In [15]:
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path
from pprint import pp

tile_size = 8
font_path = r'..\..\resources\胡晓波男神体2.0.otf'
palette_path = Path('pal1_00-9C-BD-FF.png')
optimal_font_size = []

palette = None
with Image.open(palette_path) as im:
    palette = tuple(entry[1] for entry in im.convert('L').getcolors())

pp(palette)

(0, 156, 189, 255)


In [16]:
def real_bbox(text, font_path: str, font_size: int) -> Image:
    lines = tuple(text)
    width = max([len(line) for line in lines]) * font_size
    height = len(lines) * font_size
    with Image.new('L', (width*3, height*3)) as img:
        draw = ImageDraw.Draw(img)
        font = ImageFont.truetype(font_path, font_size)
        draw.text(
            (width, height), "\n".join(lines), font=font, fill="white",
            align="center", spacing=-0.2,
            stroke_width=0.2, stroke_fill=192
        )
        
        xMin = width * 3
        yMin = height * 3
        xMax, yMax = 0, 0
        for y in range(img.height):
            for x in range(img.width):
                if (v := img.getpixel((x, y))) not in palette:
                    v = min(palette, key=lambda c: abs(c**2-v**2))
                    img.putpixel((x, y), v)
                    
                if v != 0:
                    xMin = min(xMin, x)
                    yMin = min(yMin, y)
                    xMax = max(xMax, x)
                    yMax = max(yMax, y)

        return img.crop((xMin, yMin, xMax+1, yMax+1))

In [17]:
def generate_text_img(text: str, max_columns: int, max_rows: int, font_path: str) -> Image:
    global optimal_font_size
    max_width = int(max_columns * tile_size)
    max_height = int(max_rows * tile_size)
    font_size = max(max_width, max_height)
    while True:
        img = real_bbox(text, font_path, font_size)
        if img.height > max_height or img.width > max_width:
            font_size -= 1
        else:
            print(f'{font_size = }')
            optimal_font_size.append(font_size)
            break

    print(f'width = {img.width}, height = {img.height}')
    print(f'{img.getcolors() = }')
    return img

In [18]:
text = "入门篇"
max_columns = 2
max_rows = 7

generate_text_img(text, max_columns, max_rows, font_path)

font_size = 16
width = 16, height = 51
img.getcolors() = [(421, 0), (37, 156), (116, 189), (242, 255)]


In [19]:
text = "柯南推理课堂"
max_columns = 2
max_rows = 16

generate_text_img(text, max_columns, max_rows, font_path)

font_size = 16
width = 16, height = 102
img.getcolors() = [(609, 0), (184, 156), (152, 189), (687, 255)]


In [20]:
font_size = min(optimal_font_size)
print(f'{optimal_font_size = }')

im_1 = real_bbox("入门篇", font_path, font_size)
im_2 = real_bbox("柯南推理课堂", font_path, font_size)

with Image.open("017536.png", 'r') as bg:
    bg.paste((0,0,0), (176,144,208,280))
    bg.paste(im_2, (176, 168))
    bg.paste(im_1, (208-im_1.width, 160))
    bg.save("017536_chs.png")


optimal_font_size = [16, 16]


In [21]:
print('Remember to recompress the tileset!!!')

Remember to recompress the tileset!!!
